In [1]:
from dotenv import load_dotenv
import os
import nomics
from pprint import pprint as pp
import json
from time import sleep
import pandas as pd
import requests
import io
from collections import OrderedDict
from copy import deepcopy
from tqdm.notebook import tqdm_notebook as progress_bar
from pathlib import Path

# initialize the environment
load_dotenv()
API_KEY=os.getenv("API_KEY")

nom = nomics.Nomics(API_KEY)

# get the list of currencies
currency_data_df = pd.read_pickle('..\\downloaded_data\\nomics\\crypto_data.pkl')

currency_list = currency_data_df['currency_id'].to_list()

response = {}
banned_words = ["AUX", "COM2", "COM5", "CON"]

# get list of already downloaded and pickled currencies
already_downloaded_currencies = [x.stem for x in Path('..\\downloaded_data\\nomics\\candles').glob('*.pkl')]

# get daily candles for each currency
for currency in progress_bar(currency_list, desc='Downloading candles'):
    if currency not in banned_words and currency not in already_downloaded_currencies:
        response = nom.Candles.get_candles(interval='1d', currency=currency)
        df = pd.json_normalize(response)
        df.fillna(0, inplace=True)
        # add types
        headers = df.columns.to_list()
        for column in headers:
            if column == 'timestamp':
                df[column] = pd.to_datetime(df[column], infer_datetime_format=True)
            else:
                df[column] = df[column].astype(float)
        # pickle the data
        save_path = Path('..\\downloaded_data\\nomics\\candles\\' + currency + '.pkl').resolve()
        df.to_pickle(save_path)

In [69]:
df.head()

,timestamp,open,high,low,close,volume,transparent_open,transparent_high,transparent_low,transparent_close,transparent_volume,volume_transparency.D
0,2021-10-11,0.009373,0.009373,0.009235,0.009290,2746.39,0.0,0.0,0.0,0.0,0.0,2746.0
1,2021-10-12,0.009270,0.010447,0.008641,0.009096,25964.80,0.0,0.0,0.0,0.0,0.0,25964.0
2,2021-10-13,0.009098,0.010846,0.009098,0.010448,14006.46,0.0,0.0,0.0,0.0,0.0,14006.0
3,2021-10-14,0.010500,0.010968,0.009493,0.009493,7996.77,0.0,0.0,0.0,0.0,0.0,7996.0
4,2021-10-15,0.009488,0.009488,0.009255,0.009359,2645.23,0.0,0.0,0.0,0.0,0.0,2645.0


In [2]:
import pandas as pd
adf = pd.read_pickle('..\\downloaded_data\\nomics\\crypto_data.pkl')

In [9]:
adf[adf['currency_id'] == 'GIZA']

,currency_id,price,status,name,market_cap,circulating_supply,volume,rank,high,high_timestamp,num_exchanges
4649,GIZA,127.169763,active,GizaDao,0.0,0.0,0.0,9616,0.0,0,1


In [6]:
from dotenv import load_dotenv
import os
import nomics
from pprint import pprint as pp
import json
from time import sleep
import pandas as pd
import requests
import io
from collections import OrderedDict
from copy import deepcopy
from tqdm.notebook import tqdm_notebook as progress_bar
from pathlib import Path

# initialize the environment
load_dotenv()
API_KEY=os.getenv("API_KEY")

nom = nomics.Nomics(API_KEY)

response = nom.Candles.get_candles(interval='1d', currency='GIZA')

In [7]:
pp(response)

[{'close': '131.68802947',
  'high': '279.92064986',
  'low': '131.46222595',
  'open': '131.46222595',
  'timestamp': '2021-12-04T00:00:00Z',
  'transparent_close': '131.68802947',
  'transparent_high': '279.92064986',
  'transparent_low': '131.46222595',
  'transparent_open': '131.46222595',
  'transparent_volume': '731.91',
  'volume': '731.91',
  'volume_transparency': {'A': '731'}},
 {'close': '51.21414790',
  'high': '177.09334590',
  'low': '51.21414790',
  'open': '73.89895467',
  'timestamp': '2021-12-07T00:00:00Z',
  'transparent_close': '51.21414790',
  'transparent_high': '177.09334590',
  'transparent_low': '51.21414790',
  'transparent_open': '73.89895467',
  'transparent_volume': '1783.05',
  'volume': '1783.05',
  'volume_transparency': {'A': '1783'}}]
